In [1]:
!pip3 install spacy

In [5]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/en_core_web_sm -->
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [6]:
!python3 -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/de_core_news_sm -->
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [4]:
import convokit

In [5]:
from convokit import Corpus, User, download

### Downloading dataset

In [7]:
# de = Corpus(filename=download("subreddit-de"))
germany = Corpus(filename=download("subreddit-germany"))

Dataset already exists at /home/nbuser/.convokit/downloads/subreddit-germany


KeyboardInterrupt: 

In [8]:
import os

In [10]:
os.path

<module 'posixpath' from '/home/nbuser/anaconda3_501/lib/python3.6/posixpath.py'>

In [ ]:
de = Corpus(filename=download("subreddit-de"))

Dataset already exists at /home/nbuser/.convokit/downloads/subreddit-de


#### Getting common users

In [ ]:
eng_users = set()
de_users = set()
for usr in de.iter_users():
    de_users.add(usr.name)
for usr in germany.iter_users():
    eng_users.add(usr.name)

In [ ]:
common_users = list(eng_users.intersection(de_users))
print(len(common_users))
eng_only_users = list(eng_users.difference(de_users))
print(len(eng_only_users))

In [ ]:
common_users[0]

### Topic Models

In [1]:
import re
word_pattern = re.compile("\w[\w\-\']*\w|\w")

In [8]:
def fast_and_loose_tokenizer(utterance):
    parsed = word_pattern.findall(utterance)
    return([i.lower() for i in parsed])

In [ ]:
#creates two bag of words from utterances#

cross_board_bow = []
single_board_bow = []

for utt in germany.iter_utterances():
    usr = utt.user.name
    if usr in common_users:
        for i in fast_and_loose_tokenizer(utt.text):
            cross_board_bow.append(i)
    else:
        for i in fast_and_loose_tokenizer(utt.text):
            single_board_bow.append(i)

In [9]:
#parameters#
number_topics = 5
number_words = 10
count_vectorizer = CountVectorizer(stop_words='english')

['I', 'feel', 'it', 'in', 'my', 'heart', 'and', 'soul']


['i', 'feel', 'it', 'in', 'my', 'heart', 'and', 'soul']

In [ ]:
#takes a long time#
count_data = count_vectorizer.fit_transform(cross_board_bow)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [ ]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [ ]:
#takes a lot of time#
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)


In [ ]:
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

#### Extract texts that bilingual users posted in English

In [ ]:
import re
bag = []

for utt in germany.iter_utterances():
    usr = utt.user.name
    if usr in common_users:
        line = utt.text.map(lambda x: re.sub('[,\.!?]', '', x))
        line = line.map(lambda x: x.lower()) 
        bag.append(line)


user_conv = {}
for utt in germany.iter_utterances():
    usr = utt.user.name
    if usr not in common_users or usr == '[deleted]':
        continue
    if usr not in user_conv and utt.text != '':
        user_conv[usr] = [utt.text]

    elif usr in user_conv and utt.text != '':
        user_conv[usr].append(utt.text)

In [ ]:
print(user_conv['panda527'])
print(len(user_conv))

In [ ]:
uttid_with_multi_lang_users = []
for usr in user_conv:
    for des in user_conv[usr]:
        print()

In [ ]:
print(uttid_with_multi_lang_users[:10])

In [ ]:
eng_filtered_convs = []
for conv in germany.iter_conversations():
    for uttid in conv._utterance_ids:
        eng_filtered_convs.append(uttid)

In [ ]:
print(eng_filtered_convs[:10])

#### Constructing training set
12000 bilingual users and 12000 no cross-over users

In [ ]:
import random
bi_users = list(user_conv.keys())
random.shuffle(bi_users)
bi_users = bi_users[:12000]
bi_utt = {}
for usr in bi_users:
    bi_utt[usr] = user_conv[usr]

In [ ]:
sin_utts = {}
for utt in germany.iter_utterances():
    usr = utt.user.name
    if usr not in sin_users or usr == '[deleted]':
        continue
    if usr not in sin_utts and utt.text != '':
        sin_utts[usr] = [utt.text]
    elif usr in sin_utts and utt.text != '':
        sin_utts[usr].append(utt.text)
sin_users = list(sin_utts.keys())
random.shuffle(sin_users)
sin_users = sin_users[:12000]
sin_utt = {}
for usr in sin_users:
    sin_utt[usr] = sin_utts[usr]

In [ ]:
print(bi_utt['I_Like_Spaghetti'])
print(sin_utt['EureMutter'])

In [ ]:
def produce_user_contribution_string(user: User) -> str:
    res = ""
    for utt in user.iter_utterances():
        res += utt.text + " "
    return res

In [ ]:
i = 0
for utt in germany.iter_utterances():
    if i < 100:
        print(len(produce_user_contribution_string(utt.user)))
    i += 1

In [ ]:
individual_utterances = []
big_utterance_string = ''

for usr in user_conv:
    individual_utterances.append(user_conv[usr])
    for des in user_conv[usr]:
        big_utterance_string += des
        